In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
import time
import numpy as np
from functools import partial
import itertools


In [4]:
import pydrake
import meshcat

from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import SceneGraph
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.optimization import CalcGridPointsOptions, Toppra
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.tree import RevoluteJoint
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.solvers.mosek import MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.primitives import TrajectorySource
from pydrake.trajectories import PiecewisePolynomial
from pydrake.all import Variable, Expression, RotationMatrix
from pydrake.all import MultibodyPositionToGeometryPose, ConnectMeshcatVisualizer, Role, Sphere
from pydrake.all import (
    ConvexSet, HPolyhedron, Hyperellipsoid,
    MathematicalProgram, Solve, le, IpoptSolver,
    Role, Sphere, VPolytope,
    Iris, IrisOptions, MakeIrisObstacles, Variable
)
from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff, InverseKinematics
)

import pydrake.symbolic as sym
import symbolic_parsing_helpers as symHelpers

from meshcat import Visualizer

import mcubes


# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
proc2, zmq_url2, web_url2 = start_zmq_server_as_subprocess(server_args=[])

# Sporadically need to run `pkill -f meshcat`

from symbolic_parsing_helpers import generate_rationalize_trig_expr_rules
from symbolic_parsing_helpers import xreplace
from symbolic_parsing_helpers import NotRationalFunctionException
# from symbolic_parsing_helpers import rationalize_trig_expr
from pydrake.all import RationalFunction
snopt = SnoptSolver()
def xreplace(expr, rules):
        if isinstance(expr, float) or isinstance(expr, sym.Variable):
            return expr
        assert isinstance(expr, sym.Expression), expr
        for old, new in rules:
            if expr.EqualTo(old):
                return new
        ctor, old_args = expr.Deconstruct()
        new_args = [xreplace(e, rules) for e in old_args]
        return ctor(*new_args)

def MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return HPolyhedron(query, geom, expressed_in)
def MakeFromVPolytopeOrEllipseSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return VPolytope(query, geom, expressed_in)


from pydrake.all import RationalForwardKinematics, FindBodyInTheMiddleOfChain

In [5]:
x = sym.Variable('x')
r1 = RationalFunction(sym.Polynomial(2*x+1), sym.Polynomial(x**2-1))
r2 = RationalFunction(sym.Polynomial(2*x+1), sym.Polynomial(x**2-1))
r3 = RationalFunction(sym.Polynomial(2*x+1), sym.Polynomial(x+1)*sym.Polynomial(x-1))
r4 = RationalFunction(sym.Polynomial(2*x+1), sym.Polynomial(x+1))

print(r1+r2)
print(r1+r3)
print(r1+r4)

(2*1 + 4*x) / (-1*1 + 1*x^2)
(2*1 + 4*x) / (-1*1 + 1*x^2)
(1*x + 3*x^2 + 2*x^3) / (-1*1 + -1*x + 1*x^2 + 1*x^3)


# Build simulation Environment

In [6]:
simple_collision = True
# gripper_welded = True

vis = Visualizer(zmq_url=zmq_url)
vis.delete()
vis2 = Visualizer(zmq_url=zmq_url2)
vis2.delete()

display(vis.jupyter_cell())

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

directives_file = FindResourceOrThrow("drake/sandbox/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sandbox/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False)

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)
query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))


    
ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.01)


inspector = query.inspector()
pairs = inspector.GetCollisionCandidates()


geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
HPolyhedronSets = {geom:MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}
VPolyhedronSets = {geom:MakeFromVPolytopeOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}



body_indexes_by_geom_id = {geom:
                           plant.GetBodyFromFrameId(inspector.GetFrameId(geom)).index() for geom in geom_ids} 

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/


Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6004...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/
Connected to meshcat-server.


In [7]:
def visualize_trajectory(traj):
    builder = DiagramBuilder()

    scene_graph = builder.AddSystem(SceneGraph())
    plant = MultibodyPlant(time_step=0.0)
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    parser = Parser(plant)
    parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
                "drake/manipulation/models/wsg_50_description/package.xml")))

    directives_file = FindResourceOrThrow("drake/sandbox/planar_iiwa_simple_collision_welded_gripper.yaml") \
        if simple_collision else FindResourceOrThrow("drake/sandbox/planar_iiwa_dense_collision_welded_gripper.yaml")
    directives = LoadModelDirectives(directives_file)
    models = ProcessModelDirectives(directives, plant, parser)

    q0 = [-0.2, -1.2, 1.6]
    index = 0
    for joint_index in plant.GetJointIndices(models[0].model_instance):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[index])
            index += 1

    plant.Finalize()

    to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))
    builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(plant.get_source_id()))

    traj_system = builder.AddSystem(TrajectorySource(traj))
    builder.Connect(traj_system.get_output_port(), to_pose.get_input_port())

    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)

    vis_diagram = builder.Build()

    simulator = Simulator(vis_diagram)
    meshcat.start_recording()
    simulator.AdvanceTo(traj.end_time())
    meshcat.publish_recording()
    with open("/tmp/spp_shelves.html", "w") as f:
        f.write(meshcat.vis.static_html())


# Construct Rational Kinematics

In [8]:
forward_kin = RationalForwardKinematics(plant)
q_star = np.zeros(forward_kin.t().shape[0])
t = forward_kin.t()
def convert_RationalForwardPoseList_to_TransformExpressionList(pose_list):
    ret = []
    for p in pose_list:
        ret.append(p.asRigidTransformExpr())
    return ret

def convert_t_to_q(t):
    q =np.arctan2(2*t/(1+t**2), (1-t**2)/(1+t**2))
    return q

def convert_q_to_t(t):
    q =np.tan(t/2)
    return q

def eval_cons(q0, q1, q2, c, tol):
        return 1-1*float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))
    
col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)


def eval_cons_rational(t0, t1, t2, c, tol):
        q = convert_t_to_q(np.array([t0, t1, t2])) 
        return 1-1*float(c.evaluator().CheckSatisfied(q, tol))

col_func_handle_rational = partial(eval_cons_rational, c=collision_constraint, tol=0.01)
link_poses_by_body_index_rat_pose = forward_kin.CalcLinkPoses(q_star, 
                                                         plant.world_body().index())
link_poses_by_body_index_multilinear_pose = forward_kin.CalcLinkPosesAsMultilinearPolynomials(q_star, 
                                                         plant.world_body().index())

X_WA_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)
X_WA_multilinear_list = [(r.rotation(), r.translation()) for r in link_poses_by_body_index_multilinear_pose]
geomA, geomB = next(iter(pairs))
index = FindBodyInTheMiddleOfChain(plant, body_indexes_by_geom_id[geomA], body_indexes_by_geom_id[geomB])
print(index)

BodyIndex(4)


In [9]:
q_low = plant.GetPositionLowerLimits()
q_high = plant.GetPositionUpperLimits()
t_low = forward_kin.ComputeTValue(q_low,q_star)
t_high = forward_kin.ComputeTValue(q_high,q_star)
print(t_low)
print(t_high)

[-1.73205081 -1.73205081 -1.73205081]
[1.73205081 1.73205081 1.73205081]


## Marching Cubes Obstacles

In [10]:
N = 50
vertices, triangles = mcubes.marching_cubes_func(tuple(t_low), tuple(t_high), N, N, N, col_func_handle_rational, 0.5)
vis2["collision_constraint"].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            meshcat.geometry.MeshLambertMaterial(color=0xff0000, wireframe=True))
# q = q0.copy()
# showres(q)

# Define Iris In T-Space

In [11]:
#  Now IRIS in configuration space, using dReal to solve for the growth volume
# through the nonconvex kinematics.

from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff,
)


def TangentPlane(self, point):
    a = 2 * self.A().T @ self.A() @ (point - self.center())
    a = a / np.linalg.norm(a)
    b = a.dot(point)
    return a, b

Hyperellipsoid.TangentPlane = TangentPlane


query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))

sym_plant = plant.ToSymbolic()
sym_context = sym_plant.CreateDefaultContext()

# For SNOPT test.
autodiff_plant = plant.ToAutoDiffXd()
autodiff_context = autodiff_plant.CreateDefaultContext()
snopt = SnoptSolver()


dReal_polytope_tol = 1

def CheckVolume(E, bodyA, bodyB, setA, setB, A, b, volume):
    prog = MathematicalProgram()
    q = prog.NewContinuousVariables(plant.num_positions(), "q")
    prog.AddBoundingBoxConstraint(lb, ub, q)
    prog.AddLinearConstraint(A, b-np.inf, b, q)
    p_AA =  prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    if volume < np.inf:
        prog.AddConstraint((q-E.center()).T @ E.A().T @ E.A() @ (q-E.center()) <= volume)
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)

    sym_plant.SetPositions(sym_context, q)
    X_WA = sym_plant.EvalBodyPoseInWorld(sym_context, bodyA)
    X_WB = sym_plant.EvalBodyPoseInWorld(sym_context, bodyB)
    # Add +0 pending https://github.com/RobotLocomotion/drake/issues/15216
    print(type(X_WA))
    p_WA = X_WA.multiply(p_AA+0)
    p_WB = X_WB.multiply(p_BB+0)
    prog.AddConstraint(eq(p_WA, p_WB))
    prog.SetSolverOption(dReal.id(), "precision", .9*dReal_polytope_tol)
    result = dReal.Solve(prog)
    return result.is_success(), result.GetSolution(q)


def GrowthVolume(E, bodyA, bodyB, setA, setB, A, b, guess=None):
    prog = MathematicalProgram()
    q = prog.NewContinuousVariables(plant.num_positions(), "q")

    if guess is not None:
        prog.SetInitialGuess(q, guess)

    prog.AddLinearConstraint(A, b-np.inf, b, q)
    p_AA = prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)
    prog.AddQuadraticErrorCost(E.A().T @ E.A(), E.center(), q)

    # TODO: Remove these.  They're for debugging only.
    #set_meshcat_object(vis['setA'],setA)
    #set_meshcat_object(vis['setB'],setB)
    print(type(p_AA[0]))
    if isinstance(bodyA, RigidBody_[AutoDiffXd]):
        # TODO: Update this to use MBP<double> and Jacobians if I'm going to keep it.
        def kinematics_constraint(vars):
            p_AA, p_BB, q = np.split(vars,[3,6])
            autodiff_plant.SetPositions(autodiff_context, q)
            X_WA = autodiff_plant.EvalBodyPoseInWorld(autodiff_context, bodyA)
            X_WB = autodiff_plant.EvalBodyPoseInWorld(autodiff_context, bodyB)
            p_WA = X_WA.multiply(p_AA)
            p_WB = X_WB.multiply(p_BB)
            return p_WA - p_WB
        prog.AddConstraint(kinematics_constraint, lb=[0,0,0], ub=[0,0,0],
                           vars=np.concatenate((p_AA,p_BB,q)))
        result = snopt.Solve(prog)

    else:
        # TODO: Construct these symbolic expressions once per body outside this method.
        # But I would have to substitute in the new q each time.
        # Better is to construct the prog once for each pair, and just update the cost?
        sym_plant.SetPositions(sym_context, q)
        X_WA = sym_plant.EvalBodyPoseInWorld(sym_context, bodyA)
        X_WB = sym_plant.EvalBodyPoseInWorld(sym_context, bodyB)
        # Add +0 pending https://github.com/RobotLocomotion/drake/issues/15216
        p_WA = X_WA.multiply(p_AA+0)
        
        p_WB = X_WB.multiply(p_BB+0)
        prog.AddConstraint(eq(p_WA, p_WB))
        result = snopt.Solve(prog)
        #result = dReal.Solve(prog)

    return result.is_success(), result.get_optimal_cost(), result.GetSolution(q)

def iris_cspace(query, point, require_containment_points=[], termination_threshold=2e-2, iteration_limit=100):
#     vis = Visualizer(zmq_url=zmq_url)
#     set_meshcat_object(vis['sample'], point, color=0x99dd99)

    ellipsoid_epsilon = 1e-1
    dim = plant.num_positions()
    lb = plant.GetPositionLowerLimits()
    ub = plant.GetPositionUpperLimits()
#     assert dim == 3 # need to update the volume once this changes
    volume_of_unit_sphere = 4.0*np.pi/3.0
    E = Hyperellipsoid(np.eye(3)/ellipsoid_epsilon, point)
#     set_meshcat_object(vis['ellipse'], E)
    best_volume = ellipsoid_epsilon**dim * volume_of_unit_sphere

    inspector = query.inspector()
    pairs = inspector.GetCollisionCandidates()

    P = HPolyhedron.MakeBox(lb, ub)
    A = np.vstack((P.A(), np.zeros((10*len(pairs),3))))  # allow up to 10 faces per pair.
    b = np.concatenate((P.b(), np.zeros(10*len(pairs))))

    geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
    sets = {geom:MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}

    use_autodiff = False
    use_dReal = False
    ad_bodies = {geom:autodiff_plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}
    sym_bodies = {geom:sym_plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}
    if use_autodiff:
        bodies = ad_bodies
    else:
        bodies = sym_bodies
    print
    iteration = 0
    num_faces = 2*len(lb)
    while True:
        ## Find separating hyperplanes

        for geomA, geomB in pairs:
            print(f"geomA={inspector.GetName(geomA)}, geomB={inspector.GetName(geomB)}")
            # Run snopt at the beginning
            while True:
                success, growth, qstar = GrowthVolume(E,
                    bodies[geomA], bodies[geomB],
                    sets[geomA], sets[geomB], A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, point)
                if success:
                    print(f"snopt_example={qstar}, growth = {growth}")
                    # Add a face to the polytope
                    A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                    num_faces += 1
                else:
                    break

            if use_dReal:
                tries = 0
                while True:
                    reachable, counter_example = CheckVolume(
                        E, sym_bodies[geomA], sym_bodies[geomB], sets[geomA], sets[geomB],
                        A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, np.inf)
                    if not reachable:
                        print("unreachable")
                        break
                    else:
                        z = E.A() @ (counter_example - E.center())
                        dreal_growth = z.dot(z)
                        print(f"counter_example = {counter_example}, growth = {dreal_growth}")
                        success, growth, qstar = GrowthVolume(E,
                            bodies[geomA], bodies[geomB],
                            sets[geomA], sets[geomB], A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, counter_example)
                        if success:
                            print(f"snopt_example={qstar}, growth = {growth}")
                            # Add a face to the polytope
                            A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                            num_faces += 1
                        if np.all(A[:num_faces,:] @ counter_example <= b[:num_faces] - dReal_polytope_tol):
                            # Then also add the counter-example
                            A[num_faces,:], b[num_faces] = E.TangentPlane(counter_example)
                            num_faces += 1
                    tries += 1

        if any([np.any(A[:num_faces,:] @ q > b[:num_faces]) for q in require_containment_points]):
            print("terminating because a required containment point would have not been contained")
            break

        P = HPolyhedron(A[:num_faces,:],b[:num_faces])
#         set_meshcat_object(vis[f'polytope'], P, wireframe=True)

        E = P.MaximumVolumeInscribedEllipsoid()
#         set_meshcat_object(vis[f'ellipse'], E)
        print(iteration)

        iteration += 1
        if iteration >= iteration_limit:
            break

        volume = volume_of_unit_sphere / np.linalg.det(E.A())
        if volume - best_volume <= termination_threshold:
            break
        best_volume = volume

    return P


In [12]:
def GrowthVolumeRational(E, X_WA, X_WB, setA, setB, A, b, guess=None):
    snopt = SnoptSolver()
    prog = MathematicalProgram()
    t = forward_kin.t()
    prog.AddDecisionVariables(t)
    
    if guess is not None:
        prog.SetInitialGuess(t, guess)

    prog.AddLinearConstraint(A, b-np.inf, b, t)
    p_AA = prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)
    prog.AddQuadraticErrorCost(E.A().T @ E.A(), E.center(), t)

    p_WA = X_WA.multiply(p_AA+0)

    p_WB = X_WB.multiply(p_BB+0)

    prog.AddConstraint(eq(p_WA, p_WB))
    result = snopt.Solve(prog)
    return result.is_success(), result.get_optimal_cost(), result.GetSolution(t)


In [13]:
def iris_rational_space(query, point, require_containment_points=[], termination_threshold=2e-2, iteration_limit=100):
    dReal_polytope_tol = 1e-2
    ellipsoid_epsilon = 1e-1
    dim = plant.num_positions()
    lb = plant.GetPositionLowerLimits()
    rational_lb = forward_kin.ComputeTValue(lb, q_star)
    ub = plant.GetPositionUpperLimits()
    rational_ub = forward_kin.ComputeTValue(ub, q_star)
    volume_of_unit_sphere = 4.0*np.pi/3.0
    E = Hyperellipsoid(np.eye(3)/ellipsoid_epsilon, point)
    best_volume = ellipsoid_epsilon**dim * volume_of_unit_sphere
    
    link_poses_by_body_index_rat_pose = forward_kin.CalcLinkPoses(q_star, 
                                                         plant.world_body().index())
    X_WA_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)

    inspector = query.inspector()
    pairs = inspector.GetCollisionCandidates()

    P = HPolyhedron.MakeBox(rational_lb, rational_ub)
    A = np.vstack((P.A(), np.zeros((10*len(pairs),3))))  # allow up to 10 faces per pair.
    b = np.concatenate((P.b(), np.zeros(10*len(pairs))))

    geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
    sets = {geom:MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}
    body_indexes_by_geom_id = {geom:
                               plant.GetBodyFromFrameId(inspector.GetFrameId(geom)).index() for geom in geom_ids} 
    
    #Turn onto true to certify regions using SOS at each iteration.
    certify = False
    # refine polytopes if not certified collision free
    refine = False and certify
        
    iteration = 0
    num_faces = 2*len(lb)
    while True:
        ## Find separating hyperplanes

        for geomA, geomB in pairs:
            print(f"geomA={inspector.GetName(geomA)}, geomB={inspector.GetName(geomB)}")
            # Run snopt at the beginning
            while True:
                X_WA = X_WA_list[int(body_indexes_by_geom_id[geomA])]
                X_WB = X_WA_list[int(body_indexes_by_geom_id[geomB])]
                success, growth, qstar = GrowthVolumeRational(E,
                    X_WA, X_WB,
                    sets[geomA], sets[geomB], 
                    A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, 
                    point)
                if success:
                    print(f"snopt_example={qstar}, growth = {growth}")
                    # Add a face to the polytope
                    A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                    num_faces += 1
                else:
                    break

            if certify:
                pass
            

        if any([np.any(A[:num_faces,:] @ t > b[:num_faces]) for t in require_containment_points]):
            print("terminating because a required containment point would have not been contained")
            break

        P = HPolyhedron(A[:num_faces,:],b[:num_faces])

        E = P.MaximumVolumeInscribedEllipsoid()
        print(iteration)

        iteration += 1
        if iteration >= iteration_limit:
            break

        volume = volume_of_unit_sphere / np.linalg.det(E.A())
        if volume - best_volume <= termination_threshold:
            break
        best_volume = volume

    return P

In [14]:
seed_points = np.tan(np.array([[0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]])    # out of shelf 1
                        /2)
for i in range(seed_points.shape[0]):
    vis2['seedpoints']["seedpoint"+str(i)].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0xFFB900))
    vis2['seedpoints']["seedpoint"+str(i)].set_transform(
                meshcat.transformations.translation_matrix(seed_points[i,:]))

# traj = PiecewisePolynomial.FirstOrderHold(np.array([0, 1]), np.array([seed_points[4], seed_points[1]]).T)
# visualize_trajectory(traj)

In [15]:

regions = []
ellipses = []
for i in range(seed_points.shape[0]):
    
    start_time = time.time()
    hpoly = iris_rational_space(query, seed_points[i,:], require_containment_points=[seed_points[i,:]], iteration_limit=100)
#     hpoly = iris_cspace(query, seed_points[i,:], require_containment_points=[seed_points[i,:]], iteration_limit=100)
    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
    regions.append(hpoly)
    ellipses.append(ellipse)

print("SUCCESS")

geomA=iiwa::link7, geomB=shelves::top
snopt_example=[ 0.25686152 -0.44650254  1.37305101], growth = 138.27790339845825
geomA=iiwa::link7, geomB=wsg::right_collision
geomA=wsg::left_collision, geomB=wsg::right_collision
geomA=wsg::right_collision, geomB=shelves::shelf_upper
snopt_example=[ 0.14471298 -0.66482365  0.73909564], growth = 147.06126401711242
snopt_example=[ 0.26684134 -0.53520202  0.93115289], growth = 151.49619658550938
geomA=wsg::body_collision, geomB=shelves::top
geomA=wsg::body_collision, geomB=shelves::shelf_upper
snopt_example=[ 0.32551918 -0.53379534  1.10297422], growth = 138.142516784699
geomA=wsg::left_collision, geomB=shelves::top
geomA=wsg::right_collision, geomB=shelves::top
geomA=iiwa::link7, geomB=shelves::shelf_lower
snopt_example=[ 0.5119587  -0.80097613  1.4041356 ], growth = 88.94239682323666
geomA=wsg::body_collision, geomB=shelves::shelf_lower
geomA=wsg::left_collision, geomB=shelves::shelf_lower
snopt_example=[ 0.27896158 -0.8160305   0.60048261], growt

snopt_example=[ 0.19186344 -1.23672615 -0.35546054], growth = 9.023992444050037
geomA=wsg::body_collision, geomB=shelves::top
snopt_example=[ 0.03898991 -1.0702109  -0.17559755], growth = 4.176028989953181
geomA=wsg::body_collision, geomB=shelves::shelf_upper
snopt_example=[ 0.14420559 -1.25506828 -0.25494605], growth = 3.585332366144627
geomA=wsg::left_collision, geomB=shelves::top
snopt_example=[-0.04434811 -1.22664273 -0.13630201], growth = 0.8715330056431299
snopt_example=[-0.01399985 -0.99501865  0.0385244 ], growth = 9.19832182806266
geomA=wsg::right_collision, geomB=shelves::top
geomA=iiwa::link7, geomB=shelves::shelf_lower
geomA=wsg::body_collision, geomB=shelves::shelf_lower
geomA=wsg::left_collision, geomB=shelves::shelf_lower
geomA=wsg::right_collision, geomB=shelves::shelf_lower
geomA=iiwa::link7, geomB=wsg::left_collision
geomA=iiwa::link7, geomB=shelves::shelf_upper
geomA=wsg::left_collision, geomB=shelves::shelf_upper
0
geomA=iiwa::link7, geomB=shelves::top
geomA=iiwa::l

# Choose poly to cert

In [16]:
print(regions)
poly_to_cert_safe = regions[2]
Atmp = poly_to_cert_safe.A()
btmp = poly_to_cert_safe.b()
poly_to_cert_safe = HPolyhedron(Atmp, btmp-0.3)


poly_to_cert_unsafe = regions[0]
Atmp = poly_to_cert_unsafe.A()
btmp = poly_to_cert_unsafe.b()
poly_to_cert_unsafe = HPolyhedron(Atmp, btmp+0.3)



[<pydrake.geometry.optimization.HPolyhedron object at 0x7f0f0710b930>, <pydrake.geometry.optimization.HPolyhedron object at 0x7f0f07205f70>, <pydrake.geometry.optimization.HPolyhedron object at 0x7f0f10c338f0>, <pydrake.geometry.optimization.HPolyhedron object at 0x7f0f07205e30>, <pydrake.geometry.optimization.HPolyhedron object at 0x7f0f10c20770>, <pydrake.geometry.optimization.HPolyhedron object at 0x7f0f07123af0>]


## Plot Regions

In [17]:
def inpolycheck(q0,q1,q2, A, b):
    q = np.array([q0, q1, q2])
    res = np.min(1.0*(A@q-b<=0))
    #print(res)
    return res
eps = 0.5
meshes = []
col1 = 0x002BFF
col2 = 0x3EFF00 
idx = 0
step = 1/(1.0*len(regions))
R = np.array([[1, 0, 0],[0, 0, 1],[0,1, 0]])
order = (0,1,2)
for i, region in enumerate(regions):
    A = region.A()[:,order]
    b = region.b()

    col_hand = partial(inpolycheck, A=A, b=b)
    vertices, triangles = mcubes.marching_cubes_func(tuple(t_low-eps), tuple(t_high+eps), 50, 50, 50, col_hand, 0.5)

    regstr = "regions" +str(idx)
    ellstr = "ellipse" +str(idx)
    mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
    mat.opacity = 0.3
#     vis2['regions'][regstr].set_object(
#             meshcat.geometry.TriangularMeshGeometry((R@vertices.T).T, triangles),
#             mat)
    vis2['regions'][regstr].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            mat)


    # C = ellipses[idx].A()
    # d = ellipses[idx].center()
    # radii, R = np.linalg.eig(C.T@C)
    # R[:,0] = R[:,0]*np.linalg.det(R)
    # Rot = RotationMatrix(R)
 
    # transf = RigidTransform(Rot, d)
    # mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
    # mat.opacity = 0.15
    # # vis2['ellipses'][ellstr].set_object(
    #         meshcat.geometry.Ellipsoid(np.divide(1,np.sqrt(radii))),
    #         meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True))

    # vis2['ellipses'][ellstr].set_transform(transf.GetAsMatrix4())
    
    idx+=1
A_poly = poly_to_cert_safe.A()         
b_poly = poly_to_cert_safe.b() 
col_hand = partial(inpolycheck, A=A_poly, b=b_poly)
Nhere = 75
vertices, triangles = mcubes.marching_cubes_func(tuple(t_low-eps), tuple(t_high+eps), Nhere, Nhere, Nhere, col_hand, 0.5)
mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
mat.opacity = 0.3
vis2['poly_to_cert_safe'].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            mat)
    
A_poly = poly_to_cert_unsafe.A()         
b_poly = poly_to_cert_unsafe.b() 
col_hand = partial(inpolycheck, A=A_poly, b=b_poly)
vertices, triangles = mcubes.marching_cubes_func(tuple(t_low), tuple(t_high), Nhere, Nhere, Nhere, col_hand, 0.5)
mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
mat.opacity = 0.3
vis2['poly_to_cert_unsafe'].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            mat)

In [58]:
eps = 0.5
meshes = []
col1 = 0x002BFF
col2 = 0x3EFF00 
idx = 0
step = 1/(1.0*len(regions))
R = np.array([[1, 0, 0],[0, 0, 1],[0,1, 0]])
order = (0,1,2)
cur_q = plant.GetPositions(plant.CreateDefaultContext())
bodies = {geom:plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}

for i, (geom_id,region) in enumerate(HPolyhedronSets.items()):
    regstr = "regions" +str(idx)
    print(regstr)
    
#     X_WA = plant.EvalBodyPoseInWorld(plant_context, bodies[geom_id])
#     A = region.A()@X_WA.rotation().matrix() 
#     b = region.b() - region.A()@X_WA.translation()
    
    A = region.A()
    b = region.b() 

    col_hand = partial(inpolycheck, A=A, b=b)
    n = 10
    vertices, triangles = mcubes.marching_cubes_func((-n,-n,-n), (n,n,n), 
                                                     50, 50, 50, col_hand, 0.5)

    
    mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
    mat.opacity = 0.3
    vis['regions'][regstr].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            mat)

    
    idx+=1


regions0
regions1
regions2
regions3
regions4
regions5
regions6


In [48]:
(geom_id,region) = next(iter(HPolyhedronSets.items()))
bodies = {geom:plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}
X_WA = plant.EvalBodyPoseInWorld(plant_context, bodies[geom_id])
A = region.A()@X_WA.rotation().matrix() 
b = region.b() - region.A()@X_WA.translation()
print(A)

[[ 2.29301105e-12  4.68099247e-01  8.83675899e-01]
 [ 1.00000000e+00 -4.89639659e-12 -1.14464386e-15]
 [ 4.32622963e-12  8.83675899e-01 -4.68099247e-01]
 [-2.29301105e-12 -4.68099247e-01 -8.83675899e-01]
 [-1.00000000e+00  4.89639659e-12  1.14464386e-15]
 [-4.32622963e-12 -8.83675899e-01  4.68099247e-01]]


# Get all the bodies and rational poses

In [18]:
# link_poses_by_body_index_rat_pose = forward_kin.CalcLinkPoses(q_star, 
#                                                          plant.world_body().index())
# X_WA_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)
# X_WB_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)

# inspector = query.inspector()
# pairs = inspector.GetCollisionCandidates()


# geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
# HPolyhedronSets = {geom:MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}
# VPolyhedronSets = {geom:MakeFromVPolytopeOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}



# body_indexes_by_geom_id = {geom:
#                            plant.GetBodyFromFrameId(inspector.GetFrameId(geom)).index() for geom in geom_ids} 

# geomA, geomB = next(iter(pairs))
X_WA = X_WA_list[int(body_indexes_by_geom_id[geomA])]
for i, p in enumerate(pairs):
    geomA, geomB = p
    X_WA = X_WA_list[int(body_indexes_by_geom_id[geomA])]
    X_WB = X_WA_list[int(body_indexes_by_geom_id[geomB])]
    if i >= 15:
        break
# print(X_WA)
# print(X_WB)
bodyA, bodyB = VPolyhedronSets[geomA], VPolyhedronSets[geomB]

# Choose dummy vertices

In [19]:
vertex1 = X_WA@RigidTransform(RotationMatrix(), bodyA.vertices()[:,0]).translation()
vertex2 = X_WB@RigidTransform(RotationMatrix(),bodyB.vertices()[:,0]).translation()
base_point = X_WB@RigidTransform(RotationMatrix(), bodyB.vertices().mean(axis = 1)).translation()
# print(vertex1)
# print(vertex2)
# print(bodyA.PointInSet(basePoint))

In [20]:
import numbers
def RationalFunctionFromExpression(expr):
    # TODO handle nested fractions
    LegalPolyExpressionKind = [
        sym.ExpressionKind.Var,
        sym.ExpressionKind.Add,
        sym.ExpressionKind.Mul,
        sym.ExpressionKind.Div,
        sym.ExpressionKind.Pow,
        sym.ExpressionKind.Constant
    ]
    if isinstance(expr, (numbers.Number, np.number)):
        return sym.Polynomial(expr)

    expr_kind = expr.get_kind()

    if expr.is_polynomial():
        return sym.RationalFunction(sym.Polynomial(expr))
    elif expr_kind not in LegalPolyExpressionKind:
        raise NotRationalFunctionException(expr.to_string() + " is not rational")
    elif expr_kind == sym.ExpressionKind.Div:
        (ctor, (numerator, denominator)) = expr.Deconstruct()
        numerator = sym.Polynomial(numerator)
        denominator = sym.Polynomial(denominator)
        return RationalFunction(numerator, denominator)
    elif expr_kind in LegalPolyExpressionKind:
        (ctor, args) = expr.Deconstruct()
        if expr_kind == sym.ExpressionKind.Mul or expr_kind == sym.ExpressionKind.Pow:
            res = RationalFunction(1)
        else:
            res = RationalFunction(0)
        for e in args:
            res = ctor(res, RationalFunctionFromExpression(e))
            # print(ctor, args)

        return res

    else:
        raise NotRationalFunctionException(expr.to_string() + " is not rational but of type " + expr.get_kind())

In [21]:
def convert_Expression_Vect_to_Rational(vect):
    ret = vect.copy()
    for i, v in enumerate(vect):
        ret[i] = RationalFunctionFromExpression(v)
    return np.array(ret)


# rat_vertex1 = convert_Expression_Vect_to_Rational(vertex1)
# rat_vertex2 = convert_Expression_Vect_to_Rational(vertex2)
# rat_base_point = convert_Expression_Vect_to_Rational(base_point)
# # print(rat_vert[0].denominator())

# #randomly check if expressions are equal
# tmp1 = rat_vertex1[0].ToExpression()
# tmp2 = vertex1[0]
# equal = True
# for i in range(1000):
#     test = 100*np.random.randn(len(tmp2.GetVariables()))
#     ret1 = tmp1.Evaluate(dict(zip(tmp1.GetVariables(),test)))
#     ret2 = tmp2.Evaluate(dict(zip(tmp2.GetVariables(),test)))
#     equal = equal and np.isclose(ret1,ret2)
#     if not equal:
#         print(ret1)
#         print(ret2)
#         break
# print(equal)
# print(tmp1.Expand().EqualTo(tmp2.Expand()))

In [22]:
def ExprVectIsAllConstant(vect):
    for v in vect:
        if v.get_kind() != sym.ExpressionKind.Constant:
            return False
    return True
def RatVectIsAllConstant(vect):
    for v in vect:
        tmp = v.ToExpression()
        if tmp.get_kind() != sym.ExpressionKind.Constant:
            return False
    return True
# print(ExprVectIsAllConstant(vertex1))
# print(RatVectIsAllConstant(rat_vertex1))
# print(ExprVectIsAllConstant(vertex2))
# print(RatVectIsAllConstant(rat_vertex2))

In [23]:
def RationalVectHasCommonDenominatorAcrossVect(vect):
    ret = True
    for i in range(vect.shape[0]-1):
        tmp1 = vect[i].denominator()
        tmp2 = vect[i+1].denominator()
        ret = ret and tmp1.CoefficientsAlmostEqual(tmp2, 1e-10)
        ret = tmp1.EqualTo(tmp2)
    return ret

# Certify the Regions

In [86]:
def construct_first_order_separating_hyperplane(prog, t, plane_name = ''):
    if plane_name != '':
        plane_name = '_'+plane_name
    A = prog.NewContinuousVariables(3, t.shape[0], 'A'+plane_name)
    b = prog.NewContinuousVariables(3, 'b'+plane_name)
    plane = A@t+b
    for i, p in enumerate(plane):
        plane[i] = sym.Polynomial(p)
        plane[i].SetIndeterminates(sym.Variables(t))
    return plane



def add_FirstOrder_SProd_Constraint(prog, plane_poly, t,  
                                  poly_to_cert, lagrange_mult_degree = 4, var_epsilon = None):
    A = poly_to_cert.A()
    b = poly_to_cert.b()
    n = b.shape[0]
    if var_epsilon is None:
        var_epsilon = np.zeros(n)
    constraint_poly = plane_poly
    for i in range(n):
        lagrange_poly, Q = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
        constraint_poly += lagrange_poly*sym.Polynomial(A[i,:]@t-(b[i]-var_epsilon[i]))
    constraint_poly.SetIndeterminates(sym.Variables(t))
    prog.AddSosConstraint(constraint_poly)
    return prog


def addBodyHyperplaneSide(prog, plane, VPoly, base_point, R_WA, p_WA, t, poly_to_cert, leq_or_geq,
                          lagrange_mult_degree = 4, cone_level = 1, var_epsilon = None):
    num_verts = VPoly.vertices().shape[1]
    
#     vertex_pos = R_WA@VPoly.vertices() + np.repeat(p_WA[:, np.newaxis],num_verts,1)-np.repeat(base_point[:, np.newaxis],num_verts,1)
    vertex_pos = R_WA@(VPoly.vertices())+\
                       np.repeat(p_WA[:, np.newaxis],num_verts,1)-\
                            np.repeat(base_point[:, np.newaxis],num_verts,1)

    for i, row in enumerate(vertex_pos):
        for j, v in enumerate(row):
            vertex_pos[i,j] = forward_kin.ConvertMultilinearPolynomialToRationalFunction(v)
    plane_rats = plane.dot(vertex_pos)
    plane_polys = [None for _ in plane_rats]
    for i, p in enumerate(plane_rats):
        p.SetIndeterminates(sym.Variables(t))
        
        if leq_or_geq == 'leq':
            plane_polys[i] = p.denominator()-p.numerator()
        elif leq_or_geq == 'geq':
            plane_polys[i] = p.numerator()-p.denominator()
        else:
            raise ValueError("leq_or_geq arg must be leq or geq not {}".format(leq_or_geq))
    for p in plane_polys:
        prog = add_FirstOrder_SProd_Constraint(prog, p, t, poly_to_cert, lagrange_mult_degree, var_epsilon = var_epsilon)
    

    return prog, plane_polys, plane


def add_pair_constraint(geomA, geomB, plane, prog, poly_to_cert, 
                        lagrange_mult_degree = 2, cone_level = 1, var_epsilon = None):
    index_A, index_B = body_indexes_by_geom_id[geomA], body_indexes_by_geom_id[geomB]
    VPolyA, VPolyB = VPolyhedronSets[geomA], VPolyhedronSets[geomB]
#     base_point = VPolyA.vertices().mean(axis=1)
    
    context = forward_kin.plant().CreateDefaultContext()
    forward_kin.plant().SetPositions(context, q_star)

    R_WA, p_WA = X_WA_multilinear_list[int(body_indexes_by_geom_id[geomA])]
    R_WB, p_WB  = X_WA_multilinear_list[int(body_indexes_by_geom_id[geomB])]
    
    base_point = R_WA@VPolyA.vertices().mean(axis=1)+p_WA
    
    prog, plane_polys, plane = addBodyHyperplaneSide(prog, plane, VPolyA, base_point, R_WA, p_WA , t, 
                                                     poly_to_cert, 'leq', lagrange_mult_degree, cone_level, var_epsilon)
    prog, plane_polys, plane = addBodyHyperplaneSide(prog, plane, VPolyB, base_point, R_WB, p_WB , t, 
                                                     poly_to_cert, 'geq', lagrange_mult_degree, cone_level, var_epsilon)
    return prog, plane_polys, plane

def construct_region_safety_problem(poly_to_cert, lagrange_mult_degree = 4, var_epsilon = None, plane_name = ""):
    prog = MathematicalProgram()
    prog.AddIndeterminates(t)
    plane = construct_first_order_separating_hyperplane(prog, t, plane_name = plane_name)
    for i, (geomA, geomB) in enumerate(pairs):
#         print("pair {}/{}".format(i+1, len(pairs)))
        prog, plane_polys, plane = add_pair_constraint(geomA, geomB, plane, prog, poly_to_cert,
                                                       lagrange_mult_degree = lagrange_mult_degree, var_epsilon = var_epsilon)
    return prog, plane
convSolver = MosekSolver()

prog = MathematicalProgram()
# plane = construct_first_order_separating_hyperplane(prog, t, plane_name = "{}_{}".format(geomA.get_value(), geomB.get_value()))
plane = construct_first_order_separating_hyperplane(prog, t, plane_name = "1")

prog.AddIndeterminates(t)

# prog, plane_polys, plane = add_pair_constraint(geomA, geomB, plane, prog, poly_to_cert, lagrange_mult_degree = 4)




In [105]:
def construct_first_order_separating_hyperplane(prog, t, plane_name = ''):
    if plane_name != '':
        plane_name = '_'+plane_name
    a_A_coeffs = prog.NewContinuousVariables(3, t.shape[0], 'a_A'+plane_name)
    a_b_offset = prog.NewContinuousVariables(3, 'a_b'+plane_name)
    a_poly = a_A_coeffs@t+a_b_offset
    
    b_A_coeffs = prog.NewContinuousVariables(1, t.shape[0], 'b_A'+plane_name)
    b_b_offset = prog.NewContinuousVariables(1, 'b_b'+plane_name)
    b_poly = b_A_coeffs@t+b_b_offset
    
    for i, p in enumerate(a_poly):
        a_poly[i] = sym.Polynomial(p)
        a_poly[i].SetIndeterminates(sym.Variables(t))
    b_poly = sym.Polynomial(p)
    b_poly.SetIndeterminates(sym.Variables(t))
    return a_poly, b_poly



def add_FirstOrder_SProd_Constraint(prog, plane_poly, t,  
                                  poly_to_cert, lagrange_mult_degree = 4, var_epsilon = None):
    A = poly_to_cert.A()
    b = poly_to_cert.b()
    n = b.shape[0]
    if var_epsilon is None:
        var_epsilon = np.zeros(n)
    lagrange_poly, Q = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
    constraint_poly = plane_poly+lagrange_poly
    for i in range(n):
        lagrange_poly, Q = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
        constraint_poly += lagrange_poly*sym.Polynomial(-A[i,:]@t+(b[i]-var_epsilon[i]))
    constraint_poly.SetIndeterminates(sym.Variables(t))
#     prog.AddSosConstraint(constraint_poly)
    prog.AddEqualityConstraintBetweenPolynomials(constraint_poly, sym.Polynomial(-1))
    return prog


def addBodyHyperplaneSide(prog, a_plane_poly, b_plane_poly,
                          VPoly, R_WA, p_WA, t, poly_to_cert, leq_or_geq,
                          lagrange_mult_degree = 4, cone_level = 1, var_epsilon = None):
    num_verts = VPoly.vertices().shape[1]
    
#     vertex_pos = R_WA@VPoly.vertices() + np.repeat(p_WA[:, np.newaxis],num_verts,1)-np.repeat(base_point[:, np.newaxis],num_verts,1)
    vertex_pos = R_WA@(VPoly.vertices())+\
                       np.repeat(p_WA[:, np.newaxis],num_verts,1)

    for i, row in enumerate(vertex_pos):
        for j, v in enumerate(row):
            vertex_pos[i,j] = forward_kin.ConvertMultilinearPolynomialToRationalFunction(v)
    plane_rats = a_plane_poly.dot(vertex_pos)
    plane_polys = [None for _ in plane_rats]
    for i, p in enumerate(plane_rats):
        p.SetIndeterminates(sym.Variables(t))
        
        if leq_or_geq == 'leq':
            plane_polys[i] = (b_plane_poly- 1e-8)*p.denominator()-p.numerator()
        elif leq_or_geq == 'geq':
            plane_polys[i] = p.numerator()-b_plane_poly*p.denominator()
        else:
            raise ValueError("leq_or_geq arg must be leq or geq not {}".format(leq_or_geq))
    for p in plane_polys:
        prog = add_FirstOrder_SProd_Constraint(prog, p, t, poly_to_cert, 
                                               lagrange_mult_degree, var_epsilon = var_epsilon)
    

    return prog, plane_polys, plane


def add_pair_constraint(geomA, geomB, a_plane_poly, b_plane_poly, prog, poly_to_cert, 
                        lagrange_mult_degree = 2, cone_level = 1, var_epsilon = None):
    VPolyA, VPolyB = VPolyhedronSets[geomA], VPolyhedronSets[geomB]

    R_WA, p_WA = X_WA_multilinear_list[int(body_indexes_by_geom_id[geomA])]
    R_WB, p_WB  = X_WA_multilinear_list[int(body_indexes_by_geom_id[geomB])]
    
    
    prog, plane_polys, plane = addBodyHyperplaneSide(prog, a_plane_poly, b_plane_poly, 
                                                     VPolyA, R_WA, p_WA , t, 
                                                     poly_to_cert, 'leq', lagrange_mult_degree, cone_level,
                                                     var_epsilon)
    prog, plane_polys, plane = addBodyHyperplaneSide(prog, a_plane_poly, b_plane_poly,
                                                     VPolyB, R_WB, p_WB , t, 
                                                     poly_to_cert, 'geq', lagrange_mult_degree, cone_level,
                                                     var_epsilon)
    return prog, plane_polys, plane

def construct_region_safety_problem(poly_to_cert, lagrange_mult_degree = 4, var_epsilon = None, plane_name = ""):
    prog = MathematicalProgram()
    prog.AddIndeterminates(t)
    a_poly, b_poly = construct_first_order_separating_hyperplane(prog, t, plane_name = plane_name)
    for i, (geomA, geomB) in enumerate(pairs):
#         print("pair {}/{}".format(i+1, len(pairs)))
        prog, plane_polys, plane = add_pair_constraint(geomA, geomB, plane, prog, poly_to_cert,
                                                       lagrange_mult_degree = lagrange_mult_degree, var_epsilon = var_epsilon)
    return prog, plane

In [106]:
#safe
prog = MathematicalProgram()
prog.AddIndeterminates(t)
safe = True
safe_failures = []
prog = MathematicalProgram()
prog.AddIndeterminates(t)
max_iter = 1
iter_num = 0
eps = 0
safe = False
while iter_num < max_iter:
    for i, (geomA, geomB) in enumerate(pairs):
        print("pair {}/{}".format(i+1, len(pairs)))

#         plane = construct_first_order_separating_hyperplane(prog, t, plane_name = "1")
#         prog, plane_polys, plane = add_pair_constraint(geomA, geomB, plane, prog, poly_to_cert_safe,
#                                                        lagrange_mult_degree = 8,
#                                                        var_epsilon = -eps*np.ones_like(poly_to_cert_safe.b())
#                                                       )
        a_plane_poly, b_plane_poly = construct_first_order_separating_hyperplane(prog, t, plane_name = "1")
        prog, plane_polys, plane = add_pair_constraint(geomA, geomB, a_plane_poly, b_plane_poly,
                                                       prog, poly_to_cert_safe,
                                                       lagrange_mult_degree = 4,
                                                       var_epsilon = eps*np.ones_like(poly_to_cert_safe.b())
                                                      )
    result = convSolver.Solve(prog)
    print("Var_epsilon = {}, safe = {}".format(eps, result.is_success()))
    if result.is_success():
        safe = True
        break
    else:
        eps += 0.1
        iter_num += 1
print("Safe = {}".format(safe))
    
#     if not result.is_success():
#         safe_failures += (geomA, geomB)
#     safe = safe and result.is_success()
# print(safe)

pair 1/15
pair 2/15
pair 3/15
pair 4/15
pair 5/15
pair 6/15
pair 7/15
pair 8/15
pair 9/15
pair 10/15
pair 11/15
pair 12/15
pair 13/15
pair 14/15
pair 15/15
Var_epsilon = 0, safe = False
Safe = False


In [107]:
# unsafe
prog = MathematicalProgram()
prog.AddIndeterminates(t)
safe = True
safe_failures = []
prog = MathematicalProgram()
prog.AddIndeterminates(t)

for i, (geomA, geomB) in enumerate(pairs):
    print("pair {}/{}".format(i+1, len(pairs)))
    
#     plane = construct_first_order_separating_hyperplane(prog, t, plane_name = "1")
#     prog, plane_polys, plane = add_pair_constraint(geomA, geomB, plane, prog, poly_to_cert_unsafe, 
#                                                    lagrange_mult_degree = 4,
# #                                                    var_epsilon = -0.2*np.ones_like(poly_to_cert_unsafe.b())
#                                                   )
    a_plane_poly, b_plane_poly = construct_first_order_separating_hyperplane(prog, t, plane_name = "1")
    prog, plane_polys, plane = add_pair_constraint(geomA, geomB, a_plane_poly, b_plane_poly,
                                                       prog, poly_to_cert_unsafe,
                                                       lagrange_mult_degree = 4,
                                                       var_epsilon = eps*np.ones_like(poly_to_cert_unsafe.b())
                                                      )
result = convSolver.Solve(prog)
print("Safe = {}".format(result.is_success()))


pair 1/15
pair 2/15
pair 3/15
pair 4/15
pair 5/15
pair 6/15
pair 7/15
pair 8/15
pair 9/15
pair 10/15
pair 11/15
pair 12/15
pair 13/15
pair 14/15
pair 15/15
False


In [ ]:
plane = construct_first_order_separating_hyperplane(prog, t, plane_name = "1")
print(plane)

In [ ]:

convSolver = MosekSolver()

start_time = time.time()
failures = []
for j,r in enumerate(regions):
    print("Region {}/{}".format(j, len(regions)-1))
    prog, plane = construct_region_safety_problem(r, lagrange_mult_degree = 4, plane_name=str(j))
    result = convSolver.Solve(prog)
    print("Safe: {}".format(result.is_success()))
    if not result.is_success():
        failures += [r]
end_time = time.time()
print("Certified in {}".format(end_time-start_time))

In [ ]:
result = convSolver.Solve(prog)
print(result.is_success())

In [ ]:
# DEPRECATED
# def polynomialHyperplaneConstraintPolynomial(t, leq_or_geq, rat_vertex, rat_base_point, plane):
#     vect = rat_vertex - rat_base_point
#     if not RationalVectHasCommonDenominatorAcrossVect(vect):
#         raise ValueError("vertex needs to have same denominator across entries")
#     numerator_vect = np.array([v.numerator() for v in vect])
#     prod = 0
#     for i in range(numerator_vect.shape[0]):
#         prod += numerator_vect[i]*plane[i]
#     if leq_or_geq == 'leq':
#         poly_sos_expr = -prod+vect[0].denominator()
#     elif leq_or_geq == 'geq':
#         poly_sos_expr = prod-vect[0].denominator()
#     else:
#         raise ValueError('leq_or_geq must be leq or geq not {}'.format(leq_or_geq))
#     poly_sos_expr.SetIndeterminates(sym.Variables(t))
#     return poly_sos_expr

# def polynomialHyperplaneConstraint2(prog, plane, leq_or_geq, rat_vertex, rat_base_point, t,  
#                                   t_Polyhedron_Constraint, lagrange_mult_degree = 4):
#     method_start_time = time.time()
#     start_time = time.time()
#     poly_sos_expr = polynomialHyperplaneConstraintPolynomial(t, leq_or_geq, rat_vertex, rat_base_point, plane)
#     print("Time to create poly_sos_expr: {}".format(time.time()-start_time))
#     cone_rhs, _ = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
#     A = t_Polyhedron_Constraint.A()
#     b = t_Polyhedron_Constraint.b()
#     for i in range(A.shape[0]):
#         start_time = time.time()
#         gi = sym.Polynomial(A[i,:]@t-b[i])
#         lagrange_mult_poly, Gram_matrix = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
#         cone_rhs += lagrange_mult_poly*gi
#         print("Time to add constraint {}/{}: {}".format(i+1, A.shape[0], (time.time() - start_time)))
#     start_time = time.time()
#     lagrange_mult_poly, Gram_matrix = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
#     cone_rhs += lagrange_mult_poly*poly_sos_expr
#     cone_rhs.SetIndeterminates(sym.Variables(t))
#     print("Time to add constraint poly_sos_expr: {}".format(i+1, (time.time() - start_time)))
    
#     print(cone_rhs.indeterminates())
#     print(prog.indeterminates())
#     # print(cone_rhs)
#     start_time  = time.time()
#     prog.AddEqualityConstraintBetweenPolynomials(sym.Polynomial(-1), cone_rhs)
#     print("Time to add eq constraint: {}".format(time.time() - start_time))
#     print("Time to run method {}".format(time.time() - method_start_times))
#     return prog, cone_rhs

# def polynomialHyperplaneConstraint(prog, plane, leq_or_geq, rat_vertex, rat_base_point, t,  
#                                   t_Polyhedron_Constraint, lagrange_mult_degree = 4, cone_level = 1):
#     method_start_time = time.time()
#     start_time = time.time()
#     poly_sos_expr = polynomialHyperplaneConstraintPolynomial(t, leq_or_geq, rat_vertex, rat_base_point, plane)
#     # prog = add_Positivstellensatz_polytope_infeasibility_constraint(prog, t, -poly_sos_expr, t_Polyhedron_Constraint, lagrange_mult_degree, cone_level = -1)
#     prog = add_SOS_lower_bound_constraint(prog, plane, leq_or_geq, rat_vertex, rat_base_point, t,  
#                                   t_Polyhedron_Constraint, lagrange_mult_degree = lagrange_mult_degree)
    
#     return prog
    
# def constantHyperplaneConstraint(prog, plane, leq_or_geq, vertex, base_point):
#     vect = vertex - base_point
#     if leq_or_geq == 'leq':
#         prog.AddLinearConstraint(plane@vect <= 1)
#     elif leq_or_geq == 'geq':
#         prog.AddLinearConstraint(plane@vect >= 1)
#     else:
#         raise ValueError('leq_or_geq must be leq or geq not {}'.format(leq_or_geq))
#     return prog

# def addHyperplaneConstraint(prog, plane, leq_or_geq, vertex, base_point, t, t_Polyhedron_Constraint, lagrange_mult_degree = 4):
#     if ExprVectIsAllConstant(vertex) and ExprVectIsAllConstant(base_point):
#         prog = constantHyperplaneConstraint(prog, plane, leq_or_geq, vertex, base_point)
#     else:
#         rat_vertex = convert_Expression_Vect_to_Rational(vertex)
#         rat_base_point = convert_Expression_Vect_to_Rational(base_point)
#         prog = polynomialHyperplaneConstraint(prog, plane, leq_or_geq, rat_vertex, rat_base_point, t,  
#                                   t_Polyhedron_Constraint, lagrange_mult_degree)

                                
#     return prog

# def addSeparatingHyperplaneConstraint(prog, plane, vertex1, vertex2, base_point, t, t_Polyhedron_Constraint, lagrange_mult_degree = 4):
#     # if plane_name is not None:
#     #     plane = prog.NewContinuousVariables(vertex1.shape[0], plane_name)
#     # else:
#     #     plane = prog.NewContinuousVariables(vertex1.shape[0])
#     prog = addHyperplaneConstraint(prog, plane, 'geq', vertex1, base_point, t, t_Polyhedron_Constraint, lagrange_mult_degree)
#     prog = addHyperplaneConstraint(prog, plane, 'leq', vertex2, base_point, t, t_Polyhedron_Constraint, lagrange_mult_degree)
#     return prog
    
# def new_member_of_cone_generated_by_polytope(prog, t, polytope, max_deg, cone_level = -1):
#     A = polytope.A()
#     b = polytope.b()
#     n = b.shape[0]
#     poly, _ = prog.NewSosPolynomial(sym.Variables(t), max_deg)
#     # print(type(A[0,:]@t - b))
#     fi = [sym.Polynomial(A[i,:]@t - b[i]) for i in range(n)]
#     if cone_level == -1 or cone_level > n:
#         cone_level = n
#     for lvl in range(1,cone_level+1):
#         subsets = itertools.combinations(range(n),lvl)
        
#         for s in subsets:
#             term, _ = prog.NewSosPolynomial(sym.Variables(t), max_deg)
#             cur_facts = [fi[i] for i in s]
#             for f in cur_facts:
#                 term *= f 
#             poly+= term
#     poly.SetIndeterminates(sym.Variables(t))

#     return prog, poly

# def add_Positivstellensatz_polytope_infeasibility_constraint(prog, t, poly_to_cert, polytope, max_deg, cone_level = -1):
#     prog, q1 = new_member_of_cone_generated_by_polytope(prog, t, polytope, max_deg, cone_level = cone_level)  
#     prog, q2 = new_member_of_cone_generated_by_polytope(prog, t, polytope, max_deg, cone_level = cone_level)
    
#     condition_poly = q2-q1*poly_to_cert
#     condition_poly.SetIndeterminates(sym.Variables(t))

#     prog.AddEqualityConstraintBetweenPolynomials(poly_to_cert, sym.Polynomial(-1))
#     return prog


# def add_SOS_lower_bound_constraint(prog, plane, leq_or_geq, rat_vertex, rat_base_point, t,  
#                                   t_Polyhedron_Constraint, lagrange_mult_degree = 4):
#     poly = polynomialHyperplaneConstraintPolynomial(t, leq_or_geq, rat_vertex, rat_base_point, plane)
#     A = t_Polyhedron_Constraint.A()
#     b = t_Polyhedron_Constraint.b()
#     n = b.shape[0]
#     for i in range(n):
#         lagrange_poly, Q = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
#         constraint_poly = poly + lagrange_poly*sym.Polynomial(A[i,:]@t-b[i])
#         constraint_poly.SetIndeterminates(sym.Variables(t))
#         prog.AddSosConstraint(constraint_poly)
#     return prog


# prog = MathematicalProgram()
# convSolver = MosekSolver()
# plane = construct_first_order_separating_hyperplane(prog,t,'1')

# i = 4
# tmp_polytope = HPolyhedron(poly_to_cert.A()[:i,:], poly_to_cert.b()[:i])
# rat_vertex1 = convert_Expression_Vect_to_Rational(vertex1)
# rat_base_point = convert_Expression_Vect_to_Rational(base_point)
# tmp = polynomialHyperplaneConstraintPolynomial(t, 'leq', rat_vertex1, rat_base_point, plane)
# print("starting here")
# prog.AddIndeterminates(t)
# prog = add_SOS_lower_bound_constraint(prog, plane, 'geq', rat_vertex1, rat_base_point, t,  
#                                   tmp_polytope, lagrange_mult_degree = 2)

# # cone_member = add_member_of_cone_generated_by_polytope(prog,t, tmp_polytope, 2, cone_level = -1)


# # prog = add_Positivstellensatz_polytope_infeasibility_constraint(prog, t, sym.Polynomial(1), tmp_polytope, 2, cone_level = -1)
# print("Prog start solving")
# result = convSolver.Solve(prog)
# print(result.is_success())


In [ ]:
viz_plane = False
if viz_plane:
    q0_span = np.linspace(q_low, q_high)
    q1_span = np.linspace(q_low, q_high)
    Q0, Q1 = np.meshgrid(q0_span, q1_span)
    diff = np.array([(vertex2-base_point)[i].Evaluate() for i in range(3)])
    Q2 = (1-diff[0]*Q0 - diff[1]*Q1)/diff[2]
    T1 = convert_q_to_t(Q0)
    T2 = convert_q_to_t(Q1)
    T3 = convert_q_to_t(Q2)
    coords = np.dstack([T1, T2, T3])
    for i in range(coords.shape[0]):
        for j in range(coords.shape[1]):
            t0,t1,t2 = coords[i,j,:]
            vis2['plane']["planepoint"+str(i)+str(j)].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0xFFB900))
            vis2['plane']["planepoint"+str(i)+str(j)].set_transform(
                meshcat.transformations.translation_matrix((t0,t1,t2)))


In [ ]:
def cert_safe_poly_two_body(t, poly_to_cert, X_WA, VSetA, X_WB, VSetB, lagrange_mult_degree= 6):
    prog = MathematicalProgram()
    prog.AddIndeterminates(t)
    plane = prog.NewContinuousVariables(VSetA.vertices()[:,0].shape[0])
    vertex1 = X_WA@RigidTransform(RotationMatrix(), VSetA.vertices()[:,0]).translation()
    if ExprVectIsAllConstant(vertex1):
        base_point = X_WA@RigidTransform(RotationMatrix(), VSetA.vertices().mean(axis = 1)).translation()
        order = (0,1)
    else:
        base_point = X_WB@RigidTransform(RotationMatrix(), VSetB.vertices().mean(axis = 1)).translation()
        order = (1,0)
    body_tup = VSetA, VSetB
    # safe = True
    for i in range(VSetA.vertices().shape[1]):
        vertexA =X_WA@RigidTransform(RotationMatrix(), VSetA.vertices()[:,i]).translation()
        for j in range(VSetB.vertices().shape[1]):
            vertexB = X_WB@RigidTransform(RotationMatrix(), VSetB.vertices()[:,j]).translation()
            vert_tup = vertexA, vertexB
            vertex1, vertex2 =  vert_tup[order[0]], vert_tup[order[1]]
            prog = addSeparatingHyperplaneConstraint(prog, plane, vertex1, vertex2, base_point, t, poly_to_cert, lagrange_mult_degree)
    print("Problem Formed")
    result = convSolver.Solve(prog)
    print("Problem Solved")
    safe = result.is_success()


    return safe, result

# is_safe, result = cert_safe_poly_two_body(t, poly_to_cert, X_WA, bodyA, X_WB, bodyB, lagrange_mult_degree=14)
print(is_safe)


In [ ]:
details = result.get_solver_details()

print(details.solution_status)
print(details.optimizer_time)

In [ ]:
prog = MathematicalProgram()
convSolver = MosekSolver()

# kept_regions = []
# for i, poly_to_cert in enumerate(regions):
#     safe = True
#     for geomA, geomB in pairs:
#         X_WA = X_WA_list[int(body_indexes_by_geom_id[geomA])]
#         X_WB = X_WB_list[int(body_indexes_by_geom_id[geomB])]
#         if ExprVectIsAllConstant(X_WA.translation()) or ExprVectIsAllConstant(X_WB.translation()):
#             bodyA, bodyB = VPolyhedronSets[geomA], VPolyhedronSets[geomB]
#             safe = safe and  cert_safe_poly_two_body(t, poly_to_cert, X_WA, bodyA, X_WB, bodyB, lagrange_mult_degree=4)
#         if not safe:
#             continue
#     if safe:
#         kept_regions += poly_to_cert
#     print("Completed {}/{} regions".format(i+1,len(regions)))
#     print("Kept {}/{}".format(len(kept_regions), i+1))
#     print()

In [ ]:
print(X_WA.translation()[0])

In [ ]:
# Solve path planning
if solve_path_planning:

    #visualize
    maxit = 60
    pts = []
    for it in range(maxit):
        # print(convert_t_to_q(traj.value(it*traj.end_time()/maxit)))
        pts.append(traj.value(it*traj.end_time()/maxit))
        # pts.append(traj.value(it*traj.end_time()/maxit))
        mat = meshcat.geometry.MeshLambertMaterial(color=0xFFF812)
        mat.reflectivity = 1.0
        vis2['traj']['points' + str(it)].set_object(
                    meshcat.geometry.Sphere(0.06), mat)
        vis2['traj']['points' + str(it)].set_transform(
                    meshcat.transformations.translation_matrix(pts[-1].reshape(-1,)))
        
        set_joint_angles(pts[-1].reshape(-1,))
        tf_l2 = plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(3)))
        R_l2 = tf_l2.rotation()
        tl_l2 = R_l2@np.array([0,0,0.9]) + tf_l2.translation()
        
        tf_la = plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(4)))
        R_la = tf_la.rotation()
        tl_la = R_la@np.array([0,0,1.2]) + tf_la.translation()
        
        
        mat = meshcat.geometry.MeshLambertMaterial(color=0x0029F1)
        mat.reflectivity = 1.0
        vis['traj']['link2']['points' + str(it)].set_object(
                    meshcat.geometry.Sphere(0.02), mat)
        vis['traj']['link2']['points' + str(it)].set_transform(
                    meshcat.transformations.translation_matrix(tl_l2))
        mat = meshcat.geometry.MeshLambertMaterial(color=0x07F100)
        mat.reflectivity = 1.0
        vis['traj']['linka']['points' + str(it)].set_object(
                    meshcat.geometry.Sphere(0.02), mat)
        vis['traj']['linka']['points' + str(it)].set_transform(
                    meshcat.transformations.translation_matrix(tl_la))
    

# Comments with Hongkai
1.) To reduce poly size always get transforms relative to middle link in kinematic chain between two bodies (not worried)

2.) make the hyperplane an affine function of t. Should allow plane to rotate

3.) Do line search on Ct <= b- epsilon to scale regions

4.) Rational polynomials should have coordinate degree 2. Track down this bug ASAP
